In [3]:
def CVaR_optimization(bond_data, scenario_returns, benchmark_weight, alpha=0.95):
    S, N = scenario_returns.shape
    # Create the model
    model = gp.Model("CVaR_Bond_Optimization")

    model.setParam('OutputFlag', 0)

    # Decision variables
    w = model.addVars(N, vtype=GRB.CONTINUOUS, lb=0, name="w")  # weights for bonds
    x = model.addVars(N, vtype=GRB.BINARY, name="x")  # binary selection of bonds
    VaR = model.addVar(vtype=GRB.CONTINUOUS, name="VaR")  # Value at Risk variable
    z = model.addVars(S, vtype=GRB.CONTINUOUS, lb=0, name="z")  # Auxiliary for CVaR
    
    # Objective function: Minimize CVaR risk (USING S)
    model.setObjective(
        VaR + (1 / (S * (1 - alpha))) * gp.quicksum(z[s] for s in range(S)),
        GRB.MINIMIZE
    )
    
    # Constrain the total weight
    model.addConstr(gp.quicksum(w[i] for i in range(N)) >= 0.99 , "SumToOneLower")
    model.addConstr(gp.quicksum(w[i] for i in range(N)) <= 1, "SumToOneUpper")

    # Extract expected returns from the DataFrame
    expected_returns = bond_data["ExpectedReturn"].values  # Assuming "ExpectedReturns" is a column in the DataFrame

# Add constraint in the model
    #for i in range(N):
        #if expected_returns[i] < 0:
            #model.addConstr(x[i] == 0, f"ExcludeNegativeReturn_{i}")

# Coupling weights and binary variables: w[i] = 0 if x[i] = 0
    for i in range(N):
        model.addConstr(w[i] <= x[i], f"WeightCoupling_{i}")
    
# Create array of equal weights first
    weights = np.array([1/75] * 75)  # Creates array of 75 elements each with 1/75

# Normalize to ensure they sum to exactly 1
    normalized_weights = normalize_weights(weights)

# Now use the normalized weight as benchmark
    #BenchmarkWeight = normalized_weights[0]  # Since all weights are equal, can take any element

# Deviation from benchmark
    #deviation_limit = 1
    #epsilon = 1e-2  # small tolerance

    #for i in range(N):
        #model.addConstr(w[i] >= (BenchmarkWeight - deviation_limit*BenchmarkWeight) - epsilon, f"LowerBound_{i}")
        #model.addConstr(w[i] <= (BenchmarkWeight + deviation_limit*BenchmarkWeight) + epsilon, f"UpperBound_{i}")

    #VaR constraint
    
    model.addConstr(VaR >= gp.quicksum(w[i] * scenario_returns[s, i] for i in range(N) for s in range(S)) / S, "VaR_calculation")

    #CVaR constraint
    
    for s in range(S):  # Iterate through the scenarios
        model.addConstr(z[s] >= VaR - gp.quicksum(w[i] * scenario_returns[s, i] for i in range(N)), f"cvar_constraint_{s}")

    # Liquidity Constraint
    
    liquidity_scores = bond_data["LiquidityScore"].values
    avg_liquidity = np.mean(liquidity_scores)
    min_liquidity = 0 * avg_liquidity
    model.addConstr(
        gp.quicksum(liquidity_scores[i] * w[i] for i in range(N)) >= min_liquidity,
        "MinLiquidity"
    )

    weights = np.array([1 / N] * N)
    BenchmarkWeight = weights[0]  # Equal allocation for each bond

    # Deviation from benchmark
    deviation_limit = 0.4
    for i in range(N):
        model.addConstr(w[i] >= (BenchmarkWeight - deviation_limit * BenchmarkWeight), f"LowerBound_{i}")
        model.addConstr(w[i] <= (BenchmarkWeight + deviation_limit * BenchmarkWeight), f"UpperBound_{i}")



    # Optimize the model
    model.optimize()
    
    if model.status == GRB.OPTIMAL:
        #print("Optimal solution found. List of all weights:")
        weights = [w[i].X for i in range(N)]  # Get the optimized weights for bonds
        return np.array(weights)

    if model.status == GRB.INFEASIBLE:
        print("The model is infeasible.")
        return None

0        2023-01-01
1        2023-01-01
2        2023-01-01
3        2023-01-01
4        2023-01-01
            ...    
36495    2023-12-31
36496    2023-12-31
36497    2023-12-31
36498    2023-12-31
36499    2023-12-31
Name: Date, Length: 36500, dtype: object
0       2023-01-01
1       2023-01-01
2       2023-01-01
3       2023-01-01
4       2023-01-01
           ...    
36495   2023-12-31
36496   2023-12-31
36497   2023-12-31
36498   2023-12-31
36499   2023-12-31
Name: Date, Length: 36500, dtype: datetime64[ns]
